# KoLLaVA Model from Bottom
KoVicuna 모델 가중치를 LLaVA에 적용하는 예제이다.

## 0. Requirements

In [ ]:
## Shell
pip install torch
pip install transformers
pip install fschat

## 1. KoVicuna 모델 적용하기

### 1.1. LLaVA 레포 다운로드 및 함수 정의

In [ ]:
## Shell
git clone https://github.com/haotian-liu/LLaVA.git

In [ ]:
## KoVicuna weight와 tokenizer를 적용하는 함수

import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from LLaVA.llava.model.llava import LlavaLlamaForCausalLM

def apply_delta(base_model_path, tokenizer_path, target_model_path, delta_path):
    print("Loading base model")
    base = AutoModelForCausalLM.from_pretrained(
        base_model_path, torch_dtype=torch.float16, low_cpu_mem_usage=True)

    print("Loading delta")
    delta = LlavaLlamaForCausalLM.from_pretrained(delta_path, torch_dtype=torch.float16, low_cpu_mem_usage=True)
    delta_tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

    print("Applying delta")
    for name, param in tqdm(delta.state_dict().items(), desc="Applying delta"):
        if name not in base.state_dict():
            assert name in ['model.mm_projector.weight', 'model.mm_projector.bias'], f'{name} not in base model'
            continue
        if param.data.shape == base.state_dict()[name].shape:
            param.data += base.state_dict()[name]
        else:
            assert name in ['model.embed_tokens.weight', 'lm_head.weight'], \
                f'{name} dimension mismatch: {param.data.shape} vs {base.state_dict()[name].shape}'
            bparam = base.state_dict()[name]
            param.data[:bparam.shape[0], :bparam.shape[1]] += bparam

    print("Saving target model")
    delta.save_pretrained(target_model_path)
    delta_tokenizer.save_pretrained(target_model_path)


### 1.2 Apply delta
base_model_path와 tokenizer_path의 경우 LLM 모델 (KoVicuna)로 입력한다. target_model_path의 경우 delta를 적용한 모델을 저장할 위치이다. delta_path는 LLaVA 모델을 입력한다.

In [ ]:
apply_delta(
    base_model_path="junelee/ko_vicuna_7b",
    tokenizer_path="junelee/ko_vicuna_7b",
    target_model_path="./kollava-7b-v0",
    delta_path="liuhaotian/LLaVA-7b-delta-v0"
)

## 2. 기존 LLaVA 모델 (LLaMA + CLIP)
llamba 모델을 가져와서 LLaVA 모델을 로드하는 예시이다.

### 2.1. LLaMA weight 다운로드
30GB RAM에서 작동할 수 있도록 llamba 7B 모델을 사용한다. 
Vicuna-7B를 추후에 사용하고 싶은 경우 llama-7b를 이용하여 vicuna weight를 추가하는 방법으로 얻을 수 있다.

LLaMA weights:
https://huggingface.co/decapoda-research

In [ ]:
## Shell
git lfs install
git clone https://huggingface.co/decapoda-research/llama-7b-hf

### 2.2. tokenizer_config.json 수정
다운받은 LLaMA의 경우 tokenizer_config.json을 수정해야 정상적으로 사용할 수 있다.

tokenizer_config.json 중 다음과 같이 수정하면 된다.

`"tokenizer_class": "LLaMATokenizer"` -> `"tokenizer_class": "LlamaTokenizer"`

In [ ]:
## 정상적으로 tokenizer와 model이 로드되는지 확인
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./llama-7b-hf")
model = AutoModelForCausalLM.from_pretrained("./llama-7b-hf")

### 2.3. LLaVA Apply Delta

In [ ]:
apply_delta(
    base_model_path="./llama-7b-hf",
    tokenizer_path="liuhaotian/LLaVA-7b-delta-v0",
    target_model_path="./llava-7b-v0",
    delta_path="liuhaotian/LLaVA-7b-delta-v0"
)